<a href="https://colab.research.google.com/github/mazen-Qaissi/Knowledge-Distillation/blob/main/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3)
        # print(self.pool(self.conv2))
        self.fc1 = nn.Linear(64*6*6,1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # print(x.shape)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [ ]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            # print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.data, float(correct*100) / float(batch_size*(batch_idx+1))))


fit(net,train_loader)

Epoch : 0 [0/50000 (0%)]	Loss: 2.292356	 Accuracy:0.000%
Epoch : 0 [200/50000 (0%)]	Loss: 2.330566	 Accuracy:12.255%
Epoch : 0 [400/50000 (1%)]	Loss: 2.396430	 Accuracy:13.861%
Epoch : 0 [600/50000 (1%)]	Loss: 1.659343	 Accuracy:16.722%
Epoch : 0 [800/50000 (2%)]	Loss: 2.073848	 Accuracy:17.164%
Epoch : 0 [1000/50000 (2%)]	Loss: 2.180962	 Accuracy:17.530%
Epoch : 0 [1200/50000 (2%)]	Loss: 1.636302	 Accuracy:20.017%
Epoch : 0 [1400/50000 (3%)]	Loss: 2.729067	 Accuracy:20.940%
Epoch : 0 [1600/50000 (3%)]	Loss: 2.761381	 Accuracy:22.195%
Epoch : 0 [1800/50000 (4%)]	Loss: 2.454574	 Accuracy:23.503%
Epoch : 0 [2000/50000 (4%)]	Loss: 1.472018	 Accuracy:24.501%
Epoch : 0 [2200/50000 (4%)]	Loss: 1.189066	 Accuracy:25.000%
Epoch : 0 [2400/50000 (5%)]	Loss: 2.200123	 Accuracy:25.790%
Epoch : 0 [2600/50000 (5%)]	Loss: 2.125559	 Accuracy:26.997%
Epoch : 0 [2800/50000 (6%)]	Loss: 1.674491	 Accuracy:27.318%
Epoch : 0 [3000/50000 (6%)]	Loss: 1.651280	 Accuracy:27.763%
Epoch : 0 [3200/50000 (6%)]	Loss

In [ ]:
def evaluate(model):
#model = mlp
    correct = 0 
    for test_imgs, test_labels in test_loader:
        #print(test_imgs.shape)
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*32)))

evaluate(net)